# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

In [ ]:
#The dataset contains several telemarketing efforts carried out between May 2008 and November 2010, totaling 17 marketing initiatives, according to the Materials and Methods section of the UCI paper.

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [ ]:
# Import pandas
import pandas as pd

In [ ]:
# Load the dataset from the provided path
df = pd.read_csv('data/bank-additional/bank-additional-full.csv', sep = ';')

In [ ]:
# Display the first 5 rows to confirm load success
df.head()

### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [ ]:
# Inspect info and datatypes
df.info()

# Check missing values
df.isna().sum()

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [ ]:
df.info()

# Business Objective

#To predict whether a client will subscribe to a term deposit (target y) before making a marketing call, so the bank can:

        #Reduce unnecessary calls

        #Increase efficiency

        #Improve customer targeting

#This is a binary classification problem.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [ ]:
# Copy the df to avoid altering the original
df_model = df.copy()

# Remove 'duration' as it leaks outcome info
df_model = df_model.drop(columns=['duration'])

# Convert target 'y' to binary 1/0
df_model['y'] = df_model['y'].map({'yes': 1, 'no': 0})

# Identify features
X = df_model.drop(columns=['y'])
y = df_model['y']

# Identify column types
numeric_features = X.select_dtypes(include=['int64','float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

numeric_features, categorical_features


In [ ]:
# pre processing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

preprocess = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)


### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [ ]:
from sklearn.model_selection import train_test_split

# Stratify to preserve class ratio
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42, 
    stratify=y
)

len(X_train), len(X_test)


### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

# Dummy classifier predicting majority class
baseline = DummyClassifier(strategy='most_frequent')

baseline.fit(X_train, y_train)
y_pred_base = baseline.predict(X_test)

print("Baseline Accuracy:", accuracy_score(y_test, y_pred_base))


#Baseline accuracy ≈ 0.88, since most customers say "no".

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# Build pipeline
log_reg_pipeline = Pipeline(steps=[
    ('preprocess', preprocess),
    ('model', LogisticRegression(max_iter=500))
])

# Train the model
log_reg_pipeline.fit(X_train, y_train)


### Problem 9: Score the Model

What is the accuracy of your model?

In [ ]:
y_pred_lr = log_reg_pipeline.predict(X_test)

print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [ ]:
import time
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

models = {
    "Logistic Regression": LogisticRegression(max_iter=500),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "SVM": SVC()
}

results = []

for name, model in models.items():
    # Create pipeline for each model
    clf = Pipeline(steps=[
        ('preprocess', preprocess),
        ('model', model)
    ])
    
    # Time the training
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    
    train_time = round(end - start, 4)
    train_acc = clf.score(X_train, y_train)
    test_acc = clf.score(X_test, y_test)
    
    results.append([name, train_time, train_acc, test_acc])

# Convert to DataFrame
df_results = pd.DataFrame(results, columns=["Model", "Train Time", "Train Accuracy", "Test Accuracy"])
df_results


### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.


- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

In [ ]:
#Grid Search + Hyperparameter tuning

from sklearn.model_selection import GridSearchCV

# Example: Decision Tree tuning

param_grid = {
    'model__max_depth': [3, 5, 10, None],
    'model__min_samples_split': [2, 10, 20]
}

tree_pipeline = Pipeline(steps=[
    ('preprocess', preprocess),
    ('model', DecisionTreeClassifier(random_state=42))
])

grid = GridSearchCV(tree_pipeline, param_grid, cv=3)
grid.fit(X_train, y_train)

print("Best Params:", grid.best_params_)
print("Best Score:", grid.best_score_)


In [ ]:
#  Imports

from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
)
import time

In [ ]:
# Define all four models and their hyperparameter grids
# We are wraping each model in a Pipeline with the same preprocessing

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

models_param_grids = {
    "Logistic Regression": {
        "pipeline": Pipeline(steps=[
            ("preprocess", preprocess),
            ("model", LogisticRegression(max_iter=1000, class_weight="balanced"))
        ]),
        # C controls regularization strength
        "param_grid": {
            "model__C": [0.1, 1.0, 10.0]
        }
    },
    "KNN": {
        "pipeline": Pipeline(steps=[
            ("preprocess", preprocess),
            ("model", KNeighborsClassifier())
        ]),
        # n_neighbors and weights are key KNN hyperparameters
        "param_grid": {
            "model__n_neighbors": [5, 15, 25],
            "model__weights": ["uniform", "distance"]
        }
    },
    "Decision Tree": {
        "pipeline": Pipeline(steps=[
            ("preprocess", preprocess),
            ("model", DecisionTreeClassifier(random_state=42, class_weight="balanced"))
        ]),
        # Depth and min_samples_leaf control complexity / overfitting
        "param_grid": {
            "model__max_depth": [3, 5, 10, None],
            "model__min_samples_leaf": [1, 5, 10]
        }
    },
    "SVM": {
        "pipeline": Pipeline(steps=[
            ("preprocess", preprocess),
            ("model", SVC(kernel="rbf", class_weight="balanced"))
        ]),
        # C and gamma are core SVM hyperparameters
        "param_grid": {
            "model__C": [0.1, 1.0, 10.0],
            "model__gamma": ["scale", "auto"]
        }
    }
}


In [ ]:
# Helper function to evaluate a tuned model
def evaluate_tuned_model(name, grid_search, X_test, y_test):
    """
    Prints metrics for a tuned model and returns them in a dict.
    """
    best_estimator = grid_search.best_estimator_
    
    # Use the best model to make predictions on the test set
    y_pred = best_estimator.predict(X_test)
    
    # Try to compute ROC-AUC; depending on the model we use predict_proba or decision_function
    if hasattr(best_estimator, "predict_proba"):
        y_proba = best_estimator.predict_proba(X_test)[:, 1]
        roc = roc_auc_score(y_test, y_proba)
    elif hasattr(best_estimator, "decision_function"):
        scores = best_estimator.decision_function(X_test)
        roc = roc_auc_score(y_test, scores)
    else:
        roc = float("nan")
    
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print(f"\n===== {name} =====")
    print("Best Params:", grid_search.best_params_)
    print(f"CV Best {grid_search.scoring}: {grid_search.best_score_:.3f}")
    print("--- Test Set Performance ---")
    print(f"Accuracy:  {acc:.3f}")
    print(f"Precision: {prec:.3f}")
    print(f"Recall:    {rec:.3f}")
    print(f"F1-score:  {f1:.3f}")
    print(f"ROC-AUC:   {roc:.3f}")
    
    return {
        "Model": name,
        "Best Params": grid_search.best_params_,
        "CV Best Score": grid_search.best_score_,
        "Test Accuracy": acc,
        "Test Precision": prec,
        "Test Recall": rec,
        "Test F1": f1,
        "Test ROC-AUC": roc
    }


In [ ]:
# Run GridSearchCV for all models and collect results

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring_metric = "f1"  # we optimize F1 because of class imbalance

tuned_results = []

for name, config in models_param_grids.items():
    print(f"\n### Tuning {name} ###")
    
    pipeline = config["pipeline"]
    param_grid = config["param_grid"]
    
    # Wrap the pipeline in GridSearchCV
    grid = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grid,
        scoring=scoring_metric,
        cv=cv,
        n_jobs=-1,
        verbose=1
    )
    
    # Time the grid search to compare training time
    start_time = time.time()
    grid.fit(X_train, y_train)
    end_time = time.time()
    
    print(f"Training + grid search time: {end_time - start_time:.2f} seconds")
    
    # Evaluate tuned model on the test set
    result = evaluate_tuned_model(name, grid, X_test, y_test)
    result["Train+Tune Time (s)"] = round(end_time - start_time, 2)
    tuned_results.append(result)


In [ ]:
# Summarize all tuned model results in a DataFrame

tuned_results_df = pd.DataFrame(tuned_results)
tuned_results_df

# Sort by F1-score (descending)
tuned_results_df.sort_values(by="Test F1", ascending=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Target variable distribution plot and save to PNG
plt.figure(figsize=(6, 4))
sns.countplot(x="y", data=df)
plt.title("Target Variable Distribution (y)")
plt.xlabel("Subscribed to term deposit?")
plt.ylabel("Count")

plt.tight_layout()
plt.savefig("images/target_distribution.png", dpi=300)
plt.close()  # Close figure so it doesn't show multiple times

In [ ]:
#target distribution plot
import matplotlib.pyplot as plt
import seaborn as sns

# Target variable distribution plot and save to PNG
plt.figure(figsize=(6, 4))
sns.countplot(x="y", data=df)
plt.title("Target Variable Distribution (y)")
plt.xlabel("Subscribed to term deposit?")
plt.ylabel("Count")

plt.tight_layout()
plt.savefig("images/target_distribution.png", dpi=300)
plt.close() 

In [ ]:
# Age distribution
plt.figure(figsize=(6, 4))
sns.histplot(df["age"], bins=30, kde=True)
plt.title("Age Distribution")
plt.xlabel("Age")
plt.ylabel("Count")

plt.tight_layout()
plt.savefig("images/age_distribution.png", dpi=300)
plt.close()

In [ ]:
# Proportion of "yes" by job
job_y = pd.crosstab(df["job"], df["y"], normalize="index")["yes"].sort_values(ascending=False)

plt.figure(figsize=(8, 5))
job_y.plot(kind="bar")
plt.title("Proportion of 'yes' by Job")
plt.ylabel("Proportion of yes")
plt.xlabel("Job")
plt.xticks(rotation=45, ha="right")

plt.tight_layout()
plt.savefig("images/proportion_yes_by_job.png", dpi=300)
plt.close()

In [ ]:
# model comparison plot (F1-score)

plt.figure(figsize=(7, 4))
sns.barplot(x="Model", y="Test F1", data=tuned_results_df)
plt.title("Model Comparison by F1-score (Tuned Models)")
plt.xlabel("Model")
plt.ylabel("Test F1-score")
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig("images/model_comparison_f1.png", dpi=300)
plt.close()

##### Questions